In [3]:
import os
os.chdir("C:\\Users\\ramreddymyla\\Google Drive\\01 DS ML DL NLP and AI With Python Lab Copy\\02 Lab Data\\Python")

In [4]:
# Linear Regression With Stochastic Gradient Descent for Wine Quality
from random import seed
from random import randrange
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate root mean squared error
def rmse_metric(actual, predicted):
	sum_error = 0.0
	for i in range(len(actual)):
		prediction_error = predicted[i] - actual[i]
		sum_error += (prediction_error ** 2)
	mean_error = sum_error / float(len(actual))
	return sqrt(mean_error)

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		rmse = rmse_metric(actual, predicted)
		scores.append(rmse)
	return scores

# Make a prediction with coefficients
def predict(row, coefficients):
	yhat = coefficients[0]
	for i in range(len(row)-1):
		yhat += coefficients[i + 1] * row[i]
	return yhat

# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
	coef = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			yhat = predict(row, coef)
			error = yhat - row[-1]
			coef[0] = coef[0] - l_rate * error
			for i in range(len(row)-1):
				coef[i + 1] = coef[i + 1] - l_rate * error * row[i]
			# print(l_rate, n_epoch, error)
	return coef

# Linear Regression Algorithm With Stochastic Gradient Descent
def linear_regression_sgd(train, test, l_rate, n_epoch):
	predictions = list()
	coef = coefficients_sgd(train, l_rate, n_epoch)
	for row in test:
		yhat = predict(row, coef)
		predictions.append(yhat)
	return(predictions)

# Linear Regression on wine quality dataset
seed(1)
# load and prepare data
#filename = 'winequality-white.csv'#'home.csv'
filename = 'home.csv' #'winequality-white.csv'#
dataset = load_csv(filename)
print("********************Step 1: load_csv************************")
print(dataset)

for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)
print("********************Step 2: str_column_to_float************************")
print(dataset)


# normalize
print("********************Step 3: minmax************************")
minmax = dataset_minmax(dataset)
print(minmax)
print("********************Step 4: minmax************************")
normalize_dataset(dataset, minmax)
minmax = dataset_minmax(dataset)
print(minmax,"************")
print(dataset)

# evaluate algorithm
n_folds = 5
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))
type(dataset)
#Scores: [0.14589653719313064, 0.15498850660489297, 0.11632878769162629, 0.174346541953509, 0.1806048691098013]
#Mean RMSE: 0.154

********************Step 1: load_csv************************
[['2104', '3', '399900'], ['1600', '3', '329900'], ['2400', '3', '369000'], ['1416', '2', '232000'], ['3000', '4', '539900'], ['1985', '4', '299900'], ['1534', '3', '314900'], ['1427', '3', '198999'], ['1380', '3', '212000'], ['1494', '3', '242500'], ['1940', '4', '239999'], ['2000', '3', '347000'], ['1890', '3', '329999'], ['4478', '5', '699900'], ['1268', '3', '259900'], ['2300', '4', '449900'], ['1320', '2', '299900'], ['1236', '3', '199900'], ['2609', '4', '499998'], ['3031', '4', '599000'], ['1767', '3', '252900'], ['1888', '2', '255000'], ['1604', '3', '242900'], ['1962', '4', '259900'], ['3890', '3', '573900'], ['1100', '3', '249900'], ['1458', '3', '464500'], ['2526', '3', '469000'], ['2200', '3', '475000'], ['2637', '3', '299900'], ['1839', '2', '349900'], ['1000', '1', '169900'], ['2040', '4', '314900'], ['3137', '3', '579900'], ['1811', '4', '285900'], ['1437', '3', '249900'], ['1239', '3', '229900'], ['2132', '4',

list